In [ ]:
from dcicutils import ff_utils

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

my_env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)


file_not_found = 0
deleted = 0
uploading = 0
success = 0
patch_problem = 0


files_resp = ff_utils.search_metadata('files-processed/?limit=all', key=my_auth)
files_resp2 = [i['uuid'] for i in files_resp if not i.get('genome_assembly')]
files_resp3 = [i['uuid'] for i in files_resp if not i.get('contributing_labs')]
comb_uuids = list(set(files_resp2) | set(files_resp3))
comb_resp = [i for i in files_resp if i['uuid'] in comb_uuids]
file_resp_with_exp = [i for i in comb_resp if i.get('source_experiments')]

print len(file_resp_with_exp)

In [ ]:
mapper = {
    'human':'GRCh38',
    'mouse':'GRCm38'
}

counter = 0
for a_file in file_resp_with_exp: 
    counter += 1
    if counter % 10 == 0:
        print counter
    exp = a_file['source_experiments'][0]
    exp_resp = ff_utils.get_metadata(exp, key=my_auth)
    biosample = exp_resp['biosample']
    organism = list(set([bs['individual']['organism']['display_title'] for bs in biosample['biosource']]))[0]
    g_a = mapper.get(organism)
    source_lab = exp_resp['lab']['@id']
    if not g_a:
        print(a_file['accession'], organism, 'not mapped')
        continue

    print a_file['accession'], organism, g_a, source_lab
    patch_data = {'genome_assembly': g_a, 'contributing_labs':[source_lab,]}
    ff_utils.patch_metadata(patch_data, obj_id=a_file['uuid'] , key=my_auth)